# Importing and displaying VTK data

### Import required libraries:

In [ ]:
import os
import vtk
import numpy as np
from vtk.util import numpy_support
import k3d
from k3d.helpers import download
from ipywidgets import interact, interact_manual, interactive, HBox, VBox
from ipywidgets import widgets, Layout
import matplotlib.pyplot as pl

### Initialize user interface:

In [ ]:
#cms = [attr for attr in dir(k3d.colormaps.matplotlib_color_maps) if not attr.startswith('__')]
cms = ['RdBu','PuOr','Blues','RdYlGn','RdGy','YlOrBr']
vtpdir = 'https://raw.githubusercontent.com/UK-Digital-Heart-Project/JupyterVTK/master/data/'
vtp_dict_urls = {'LVProperty1':'exampleLV_wallthick.vtp', 
                 'LVProperty2':'exampleLV_distprof.vtp',
                 'LVProperty3':'exampleLV_Property3.vtp'}
vtp_dict = dict()
for ky in list(vtp_dict_urls.keys()):
    filename = download(vtpdir+vtp_dict_urls[ky])
    LVreader = vtk.vtkXMLPolyDataReader()
    LVreader.SetFileName(filename)
    LVreader.Update()
    vtp_dict[ky]=LVreader.GetOutput()
vtp_dict_keys = list(vtp_dict.keys())

plot2 = k3d.plot(height=400)
plot2.wireframe=True
plot2.display()

maps =widgets.Dropdown(options=vtp_dict_keys, value=vtp_dict_keys[0], description='Mesh:')
cmsel=widgets.Select(options=cms,value=cms[0],description='ColorMap', rows=len(cms))

#@interact(x=maps, y=cmsel)
def plot(x, y):
    sh2 = vtp_dict[x]
    scvtk = sh2.GetPointData().GetScalars()
    sc = numpy_support.vtk_to_numpy(scvtk)
    scmin, scmax = np.min(sc), np.max(sc)

    global plot2
    numobj=len(plot2.objects)
    if numobj>0:
        for i in range(numobj): plot2 -= plot2.objects[-1]

    LVmesh = k3d.vtk_poly_data(sh2,
                             color_attribute=(scvtk.GetName(),scmin, scmax),
                             color_map=getattr(k3d.matplotlib_color_maps, cmsel.value),
                             color_range=[scmin, scmax])
    plot2 += LVmesh

    lvobj=plot2.objects[0]
    lvobj.color_map = getattr(k3d.matplotlib_color_maps, y)
    a = np.array([lvobj.color_range])
    pl.figure(figsize=(9, 0.5))
    img = pl.imshow(a, cmap=y)
    pl.gca().set_visible(False)
    cax = pl.axes([0.1, 0.2, 0.8, 0.4])
    pl.colorbar(orientation="horizontal", cax=cax)

out = widgets.interactive_output(f=plot, controls={'x':maps, 'y':cmsel})
widgets.VBox(children=[out, widgets.HBox(children=[maps,cmsel])])